In [33]:
from pathlib import Path
import torch, numpy as np, pandas as pd
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import v2
import timm

In [34]:
device = torch.device("cuda")

In [35]:
data_path = Path() / 'paddy-disease-classification' / 'train_images'
data = ImageFolder(data_path, transforms.Compose([
    v2.Resize((480, 480)),
    transforms.ToTensor(),
    ]))

In [36]:
train_data, valid_data = random_split(data, [0.8, 0.2])

train_dls = DataLoader(train_data, batch_size=64)
valid_dls = DataLoader(valid_data, batch_size=64)

In [37]:
model = timm.create_model('resnet26d')

In [38]:
learning_rate = 1e-3
batch_size = 64
epochs = 5
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [39]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # back-propagate
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}, [{current:>5d}/{size:>5d}]")

In [40]:
def valid_loop(dataloader, model, loss_fn):
    model.eval()
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test error: \n Accuracy: {(100 * correct):>0.1f}, Avg Loss: {test_loss:>8f} \n")


In [41]:
model = model.to(device)

In [42]:
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------------")
    train_loop(train_dls, model, loss_fn, optimizer)
    valid_loop(valid_dls, model, loss_fn)
print("Done!")

Epoch 1
---------------------------
loss: 7.011763, [   64/ 8326]
loss: 3.480344, [ 6464/ 8326]
Test error: 
 Accuracy: 25.9, Avg Loss: 3.160869 

Epoch 2
---------------------------
loss: 3.214124, [   64/ 8326]
loss: 2.423361, [ 6464/ 8326]
Test error: 
 Accuracy: 28.8, Avg Loss: 2.433945 

Epoch 3
---------------------------
loss: 2.437737, [   64/ 8326]
loss: 2.133567, [ 6464/ 8326]
Test error: 
 Accuracy: 30.8, Avg Loss: 2.177064 

Epoch 4
---------------------------
loss: 2.205904, [   64/ 8326]
loss: 1.983866, [ 6464/ 8326]
Test error: 
 Accuracy: 32.5, Avg Loss: 2.046982 

Epoch 5
---------------------------
loss: 2.089398, [   64/ 8326]
loss: 1.894946, [ 6464/ 8326]
Test error: 
 Accuracy: 33.8, Avg Loss: 1.972178 

Done!


In [43]:
torch.save(model, "paddies-model.pth")